# RPC, Pointers and Computational Graphs - Session 4

<blockquote>
"Your focus determines your reality." ~ Qui-Gon Jinn</blockquote>

In this lesson we cover:  
- aa


## Remote Procedure Call

To achieve remote data science, execution needs to be done at a distance. In its more general form this is usually called Remote Procedure Call or RPC. Instead of a program already existing on the computer which holds private data, a remote `client` can send the program in, possibly one step at a time and after checking various security rules, the server or `domain` can execute the program against its local data, storing the result of each computation inside the `domain`.

By doing this, we are able to control a variety of aspects of remote execution such as only allowing certain commands to run when issued by certain users on certain data.

### Messages

To provide a generic mechanism with which to issue commands or receive responses, `syft` has an underlying `message` class. As mentioned in earlier sessions all communication between a `client` and a node such as a `domain` or `network` is done via a series of `messages`.

The following code has been abbreviated for clarity:

```python
# packages/syft/src/syft/core/common/message.py

class SyftMessage:
    msg_id: UID
    address: Address
    
    def sign(self, signing_key: SigningKey) -> SignedMessage:
    
class SignedMessage(SyftMessage):
    obj_type: str
    signature: bytes
    verify_key: VerifyKey
    message: bytes
```

## Signing

The important aspect of a message is that it has an `address` and its own `UID` our `uuid` object. To ensure the validity of messages we make sure that they are signed. The `SyftMessage` parent class has a method `sign` which uses the supplied `signing_key` to generate a signature hash of the contents of the message `bytes` and includes a copy of the derived `verify_key`. Together this allows anyone to verify the message was signed by the owner of the `signing_key` and has never been altered.

Remember, a signature is not the same thing as `encryption`, the contents of the message are readable by anyone who can `deserialize` the bytes.

However, if a node finds that the signature on a received message does not match the attached key, it will throw the message away.

## Sync vs Async

Generally speaking if a message has a `reply_to` address it is a `sync` message which expects a response to be sent back to the requester, where as if the message has no reply address it expects no response and will be sent to the `queue` and executed by the `worker`.

There are a number of classes used to divide messages into the `WithReply` and `WithoutReply` types which are used in conditional statements to direct the flow of execution.

The most common and simple concrete `message` types are the `Immediate` messages which are intended for immediate execution on arrival.

The following code has been abbreviated for clarity:

```python
# packages/syft/src/syft/core/common/message.py

class ImmediateSyftMessageWithoutReply(ImmediateSyftMessage, SyftMessageWithoutReply):
    signed_type = SignedImmediateSyftMessageWithoutReply

class ImmediateSyftMessageWithReply(ImmediateSyftMessage, SyftMessageWithReply):
    signed_type = SignedImmediateSyftMessageWithReply
    reply_to: Address
```

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

## Actions vs Services

Messages are further divided up into a few different subclasses with slightly different semantics which we generally refer to as Actions and Services.

Actions are what we call messages which create and then act on data within the `key-value` store.

Two very common actions are `FunctionOrConstructorAction` or `RunClassMethodAction` which work to allow creation of data or execution of methods on data within the store.


## Actions

Here we can see an example Action where we are creating some new data, in this case an `integer`.

The Action might look something like `FunctionOrConstructorAction` and have a `path` to the init function for integers, such as `syft.lib.python.Int.__init__`.

The action will also contain some data for the `*args` and `**kwargs` in this case the literal `1`.

The sender of the `Action` also defines a `UUID` key where they wish the result to be stored, in this case we will use a single letter such as `A` for the UUID to make things easier to read.

We can represent that action visually like so:

<img width="300" src="data:image/webp;base64,
UklGRnQ3AABXRUJQVlA4WAoAAAAkAAAAOwUAnQEASUNDUPgPAAAAAA/4YXBwbAIQ
AABtbnRyUkdCIFhZWiAH5gAHAAoADwAVAAdhY3NwQVBQTAAAAABBUFBMAAAAAAAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLWFwcGwAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABJkZXNjAAABXAAAAGJkc2NtAAABwAAA
BJxjcHJ0AAAGXAAAACN3dHB0AAAGgAAAABRyWFlaAAAGlAAAABRnWFlaAAAGqAAA
ABRiWFlaAAAGvAAAABRyVFJDAAAG0AAACAxhYXJnAAAO3AAAACB2Y2d0AAAO/AAA
ADBuZGluAAAPLAAAAD5jaGFkAAAPbAAAACxtbW9kAAAPmAAAACh2Y2dwAAAPwAAA
ADhiVFJDAAAG0AAACAxnVFJDAAAG0AAACAxhYWJnAAAO3AAAACBhYWdnAAAO3AAA
ACBkZXNjAAAAAAAAAAhEaXNwbGF5AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAbWx1YwAAAAAAAAAmAAAADGhySFIAAAAUAAAB2GtvS1IAAAAMAAAB7G5i
Tk8AAAASAAAB+GlkAAAAAAASAAACCmh1SFUAAAAUAAACHGNzQ1oAAAAWAAACMGRh
REsAAAAcAAACRm5sTkwAAAAWAAACYmZpRkkAAAAQAAACeGl0SVQAAAAYAAACiGVz
RVMAAAAWAAACoHJvUk8AAAASAAACtmZyQ0EAAAAWAAACyGFyAAAAAAAUAAAC3nVr
VUEAAAAcAAAC8mhlSUwAAAAWAAADDnpoVFcAAAAKAAADJHZpVk4AAAAOAAADLnNr
U0sAAAAWAAADPHpoQ04AAAAKAAADJHJ1UlUAAAAkAAADUmVuR0IAAAAUAAADdmZy
RlIAAAAWAAADim1zAAAAAAASAAADoGhpSU4AAAASAAADsnRoVEgAAAAMAAADxGNh
RVMAAAAYAAAD0GVuQVUAAAAUAAADdmVzWEwAAAASAAACtmRlREUAAAAQAAAD6GVu
VVMAAAASAAAD+HB0QlIAAAAYAAAECnBsUEwAAAASAAAEImVsR1IAAAAiAAAENHN2
U0UAAAAQAAAEVnRyVFIAAAAUAAAEZnB0UFQAAAAWAAAEemphSlAAAAAMAAAEkABM
AEMARAAgAHUAIABiAG8AagBpzuy37AAgAEwAQwBEAEYAYQByAGcAZQAtAEwAQwBE
AEwAQwBEACAAVwBhAHIAbgBhAFMAegDtAG4AZQBzACAATABDAEQAQgBhAHIAZQB2
AG4A/QAgAEwAQwBEAEwAQwBEAC0AZgBhAHIAdgBlAHMAawDmAHIAbQBLAGwAZQB1
AHIAZQBuAC0ATABDAEQAVgDkAHIAaQAtAEwAQwBEAEwAQwBEACAAYQAgAGMAbwBs
AG8AcgBpAEwAQwBEACAAYQAgAGMAbwBsAG8AcgBMAEMARAAgAGMAbwBsAG8AcgBB
AEMATAAgAGMAbwB1AGwAZQB1AHIgDwBMAEMARAAgBkUGRAZIBkYGKQQaBD4EOwRM
BD4EQAQ+BDIEOAQ5ACAATABDAEQgDwBMAEMARAAgBeYF0QXiBdUF4AXZX2mCcgBM
AEMARABMAEMARAAgAE0A4AB1AEYAYQByAGUAYgBuAP0AIABMAEMARAQmBDIENQRC
BD0EPgQ5ACAEFgQaAC0ENAQ4BEEEPwQ7BDUEOQBDAG8AbABvAHUAcgAgAEwAQwBE
AEwAQwBEACAAYwBvAHUAbABlAHUAcgBXAGEAcgBuAGEAIABMAEMARAkwCQIJFwlA
CSgAIABMAEMARABMAEMARAAgDioONQBMAEMARAAgAGUAbgAgAGMAbwBsAG8AcgBG
AGEAcgBiAC0ATABDAEQAQwBvAGwAbwByACAATABDAEQATABDAEQAIABDAG8AbABv
AHIAaQBkAG8ASwBvAGwAbwByACAATABDAEQDiAOzA8cDwQPJA7wDtwAgA78DuAPM
A70DtwAgAEwAQwBEAEYA5AByAGcALQBMAEMARABSAGUAbgBrAGwAaQAgAEwAQwBE
AEwAQwBEACAAYQAgAGMAbwByAGUAczCrMOkw/ABMAEMARHRleHQAAAAAQ29weXJp
Z2h0IEFwcGxlIEluYy4sIDIwMjIAAFhZWiAAAAAAAADzFgABAAAAARbKWFlaIAAA
AAAAAIOeAAA9tP///7tYWVogAAAAAAAAS7oAALOLAAAK11hZWiAAAAAAAAAnfQAA
DsEAAMibY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAoAC0AMgA2ADsAQABF
AEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCfAKMAqACtALIAtwC8
AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEfASUBKwEyATgBPgFF
AUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJAdEB2QHhAekB8gH6
AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKiAqwCtgLBAssC1QLg
AusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOuA7oDxwPTA+AD7AP5
BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTwBP4FDQUcBSsFOgVJ
BVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZqBnsGjAadBq8GwAbR
BuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgfCDIIRghaCG4IggiW
CKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woRCicKPQpUCmoKgQqY
Cq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxDDFwMdQyODKcMwAzZ
DPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62DtIO7g8JDyUPQQ9e
D3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFtEYwRqhHJEegSBxIm
EkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRqFIsUrRTOFPAVEhU0
FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReuF9IX9xgbGEAYZRiK
GK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7G2MbihuyG9ocAhwq
HFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8THz4faR+UH78f6iAV
IEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4I2YjlCPCI/AkHyRN
JHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneierJ9woDSg/KHEooijU
KQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxuLKIs1y0MLUEtdi2r
LeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGCMbox8jIqMmMymzLU
Mw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbpNyQ3YDecN9c4FDhQ
OIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTykPOM9Ij1hPaE94D4g
PmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1QvdDOkN9Q8BEA0RH
RIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kdSWNJqUnwSjdKfUrE
SwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/dUCdQcVC7UQZRUFGb
UeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3V0RXklfgWC9YfVjL
WRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5sXr1fD19hX7NgBWBX
YKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9ZpJm6Gc9Z5Nn6Wg/
aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5rbsRvHm94b9FwK3CG
cOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4d1Z3s3gReG54zHkq
eYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/lgEeAqIEKgWuBzYIw
gpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokziZmJ/opkisqLMIuW
i/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLjk02TtpQglIqU9JVf
lcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3nWSd0p5Anq6fHZ+L
n/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adup+CoUqjEqTepqaoc
qo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJLssKzOLOutCW0nLUT
tYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2Pvgq+hL7/v3q/9cBw
wOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6ybnKOMq3yzbLtsw1
zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO1dHWVdbY11zX4Nhk
2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM4lPi2+Nj4+vkc+T8
5YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO6070DvzPBY8OXxcvH/
8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH/Jj9Kf26/kv+3P9t
//9wYXJhAAAAAAADAAAAAmZmAADypwAADVkAABPQAAAKW3ZjZ3QAAAAAAAAAAQAB
AAAAAAAAAAEAAAABAAAAAAAAAAEAAAABAAAAAAAAAAEAAG5kaW4AAAAAAAAANgAA
rgAAAFIAAABDwAAAsMAAACZAAAANwAAAUAAAAFRAAAIzMwACMzMAAjMzAAAAAAAA
AABzZjMyAAAAAAABDHIAAAX4///zHQAAB7oAAP1y///7nf///aQAAAPZAADAcW1t
b2QAAAAAAAAGEAAAoEQAAAAA2ZNdgAAAAAAAAAAAAAAAAAAAAAB2Y2dwAAAAAAAD
AAAAAmZmAAMAAAACZmYAAwAAAAJmZgAAAAIzMzQAAAAAAjMzNAAAAAACMzM0AFZQ
OCBMIwAAMPIAnQEqPAWeAT4xGItEIiGhEHh0pCADBLS3dzBSoYu6UT31vfWGlVWQ
H/NPkpQeIf53+QHfX/V/6z+2/9m/6Xmb+R/rX5S/23/xe/7/Vd9vojzN/iX1u/C/
2H9vP73+4P3r/af9Z/VfFXgBfiv8k/xv9Q/a7/D/t57ieyvhr9gX3R+wf6X+6/vL
/i/R//lfzI9yvsN/uP7T8AH88/pX+Z/r371/375j/7Pg0ff/+37AX9A/pv+j/sn+
O/8n+W+l799/5P+I/yn7h+1P86/un/O/wn+t/bT7DP5X/Wf93/d/87/8v9N/////
90P/39vX7Wex5+s//Z/P8NJHaIQjpBi2hBZUkrx/kSs4VDFtCCypxV1gS6h9QxbQ
gsqcVdYEuofUMW0ILKnFXWBLqH1DFtCCypxV1gS6h9QxYTDhBviehYPU73fSVkEM
6LccohjHKy6M9kRdR2EDreYTGa6+6EdCPGfrnONEqD/xofRXONEt7z7Ss2rnGiW9
59pWbVzjRLe8+0rNoXjeJLJZtVHn///nSu3x7wvozK9+4KyQ7fHkkKfZtGiS3uzp
dq5xolvefaVm1c40S3vPtKzaucaJb3n2ORPeb/tsKr9HkIoMFF2q/ic5HBXoP6BF
4XyPQGECuDRORr6bU/+R5kY75RPfyi7VzjRLe8+0rNq5xolvefaVm0Lg4LI/N9DP
9J5Rdq5xoeruQBiRirH59UaI+yKY9ArNq5xolvefaVm1c40S3vPtKzaucaJb0+Nf
c/pnuyXkCie/lF2rnGcIZgmKuuakdPeb9DXip9Te+VoI3bQ0VzjRLe8+0rNq5xol
vefaVm1c40S3vPtGF1YuIULHJkxolvefaVm1ccrXPy+njNpJsIALg30tpWbVzjRL
e8+0rNq5xolvefaVm1c40PDgWyAkIfZtXONEt7z7SnE51LC02NCL+UFDFolvefaV
m1c40S3vPtKzaucaJb3n2lORBlNt+lF2rnGiW959pUK5bLNa1waEX8oKGLRLe8+0
rNq5xolvefaVm1c40S3vPtGeWnzwFdxzGiW959pWbVzjQ7P93HeMuBCe/YiucaJb
3n2lZtXONEt7z7Ss2rnGiW958Ca8N8zXre8+0rHcmKleX2RGz+5flF2rjgluYWzm
t0Cs2jRJb3n2lZtXONEpb+aMvruK9HV07+KJnmBNAkAd0c9a9zotGes8BTblBlTJ
AaPumBZoYv90k9lEt7z7SsdqUnQacxLe8+0ajK1twYoceNPI++daoUXaucVYrBPf
58K5xW3VXONEt7z7Ss2jMQvvvAEnJsmTGLZ2e6frVfkdm5iqX+lqikLYsgQwCSQD
mSooD+SBK4fF9m1c40PKVopDv5RdquoBltzf/8LpnBdLKsfWlWjS4UXaubajQIJh
Ce/YiucaJb3n2lZtXIGsn4YZJrlS+j5W1ShmxcCU5CnZKFikrKt1GO8OAVCFE9/K
Lpj7/WCi7Vx5dqIvbGhTi5pUCbHVZI0y0S3vNZp4x2Eo3vPsZTRPfyi7VzjRLemD
l8dgQqBOLdYtQhvaGoTPi4odLLO5sC0tbQH7piPn+ps2oz5Rdq5xVHU6q3n2lRIx
/TqXjjwFnQrvvDxe2omjwaJb3n2JhXe4baUTzqKCie/lF2rnGiUpUDf4uU1O7p7H
zohYaYf9ghZkDWTcr3ozqOGJDMDiPntBhhSvPtKzauO/TROz0onv1zyB5VC0gRQo
6mCVXxt+Y9FV2rnGh6WChx7aUTzqKCie/lF2rnGiW959pWesAjHhtJxoRLS0S3vP
iD0Mp8GWlolvibtKzaucZxi6My1JQKzaNElvefaVm1c40S3vPtKzaucaJb3n2lZp
F0ilI9tKJ7+UXaucaJb3Xcyou7onmsIv5QUMWiW959pWbVzjRLe8+0rNq5xolvef
aNXUqlhlsE+0rNq5xolvefaMIhNRklKQi/lBQxaJb3n2lZtXONEt7z7Ss2rnGiW9
59pTkGXQ7fl5aWiW959pWbVzaYHV6XYZH/jQ+iucaJb3n2lZtXONEt7z7Ss2rnGi
W959iRdBG4NFFFyie/lF2rnGiUjGrKHDQWnm6BWbRokt7z7Ss2rnGiW959pWbVzj
RLe8+0rNqtMdHhjocpIY5RPfyi7VzjQ8ZKqE4EihzbEVzjOgi/lF2rnGiW959pWb
VzjRLe8+0rNq5xofYTnku9UoriqXlc40S3vPtKdqwAsNCGLadfS9Vc2/e8+0rNq5
xolvefaVm1c40S3vPtKzaucZwTQ9IPLcMzOVZxolvefaMwe7owI9g1VNzRYEY75H
oFZtXONEt7z7Ss2rnGiW959pWbVzjRLe8+BZ0nKtepSg4O4VnxolvdbAjGzq3u0V
yzJ/O1cgponvPwrnGiW959pWbVzjRLe8+0rNq5xolvefaVml5MsfuKVJwnnVljEo
UT3j+TYuad421/9Acblc6CiWIb+SdkbvUILKnDYa0bJqH1DFtCCypxV1gS6h9Qxb
QgsqcVdYEuofUMW0ILKnFXWBLqH1DFtCCypxV1gS6h9QwKgF9Mw4JRxtZjU9OSFz
5N1M9dXXEttQgu7kOEZXT9Ltwa9MhZoM99ipoi7VxoAA/vb8G0qYZhdo658oH5iZ
T/CT1xU7JP0chP3eS/aA6lvyjQ7dRGRunNijDOktc3NT2aShTLtfGQwiF0iP7SHa
PBtedv8o64YHPevHllEbONgM9xJC4NAAfaIFErXtjeaarSg79ClDgmm3T561f/4W
pQtUlZXTuTe3dT3PjMxKqqdwsrsiZdajKpxA73pJOXo3b085Vl3R/CGAxYh0pk7V
Zi2RHmkJqhjiS4zlwaNndpD2kG1x/uTZ1HBjtKn2OJonk+WE3FWz23vXo+AN/qQ7
gYWlmP23SYuwfnYyu4McRPSx1qJgHWf+AzSrfU20eDo4WNzbZdP0MUjqzc0+3OSx
OZ2lJ+tzHgsnwPTdNZQd3/MuEKt165wZqTqrN7ex4dpxpzjJqvTewtKUup853ARE
JiB4m6XmIWxgrrrroLNAbMKEQeEoJK4q0/Yhoa1wupV329KTSQioWjvWB60BylM7
aaMPEFJHom9xZ3lr54mVxgoRZNP2BdNuiZjT85WV2/KP2RNDW0tuX/w+MKTWOq4G
NnHdTiEN9zwLfE3BGndq6SSdYBiMAWwN/7GLuUHDqlRHG2AT3bMer5B9AEmui4fF
PnMrhMwPLSjjqMm+3GmAybBysxW5KF3RmA+WJzX//As3ol2b1cD01DeRKB2Zv/9D
RExvjjD/3cQjSm8OuUaAi9qH64NXCcgLpfnYG+4LEgeaK1cC368in3BM3xarwRGp
gva8GxaYtMVZ9G9egAZEa2MItl0XYCidlDUyDVuwy28sS1tqEiQ3QV/boeF1XCeD
CL0wq3pwCX+Ri0W2uJCp/fG7+r9VTNE8RG4RMVYpuCu2xM2thUH+vBB6yg6/8PsU
iFuuscVnnvgiXwl6qW1j69F0bp4UmekIP4FW4s2gqcuLJ2/ZO8JnqFA8gGfeZpcp
hrmoPk+Dtaxq5De1cf4zGFPllb/lQAZO1bGWGV5Y5DFdHxZDVNQfn5uc6iwitukX
BCRDdhhIjL7NhBGg9JepIk06iOAoEKX2BUnu8SKTINAM/PwRNaipffze17c3dcLt
QGDbrrYZ76v3oSpvgOUC1O7n3tZsh3FoGkEv9h+n31Jgny1kM7+VIS4yly4ar6qR
M0EsI16AAOSF0gyYTpD4MMkgnakQZBLomAq0LlMvUTbYHNOwZSBLf7V3ZPGyRNwq
0ozGvXyWGUj5l7wfgi6cA+Xqo1wclatGLY9eDB1EGKITvyeTsPe5RmK2LEU8rpT9
DnUw5rWARSpKlKvd67FD4B53rpuAQonuETA7LYMe+905o+cVbRbF4cAAW/o2f/62
pZ26VP9HJirB976bPzopOyarRox6OmNQgVIgTq7enBL0hmOeK5dSa/KZuD/yXc+5
W78awMOI7a4sOVd/qAcxdOGQvaiXJYMcIyLcsieMM6shYTjzJPFsTRfWjkyuFHrC
dyAAUK+IKaGhI8cnYEib78jGQ+WPB3kaAscB5/fTbBKhGD+pclfJ9Wgk9PMWxvA5
5svzrNN3XbqbYYtklcUSNCnlFI5a1XS06I7B4a91PGov50fHLAhOjv2mvWldcYzS
ooABHNuhwn/4TVirQC2MUcfD82YA2Eq/Au2YmRWJvTWVMLAd3AddgPNWphMKCuXN
YHKA5dsaFbkTYlgovLS1ovCyMFIoR2P2cHJGIXcYVnF62hm2Y6XejxWhx2dUn/TG
dk1Vv/GMAGvh3SjXRFdvxGp2qGII38+KWgYyvi5vKdQORWVHcmD+ax2jkfpljs+z
DCqVWt5uK1cBWWiyY5DJ0RCh5iUW5OOige12aqci1LKvjZ285PVUf84NKW9PcJv9
OIpiagCARS7Hfl4iHmlsx4EGmvvgN10Nojn2cJB23G25vu0u/vlF4BpcXTIEmlFq
gzYCi9THhoWxnQaIhr/0gWPYNQt5MxT0gEg1CGatZBC/f1ZCqZWyOMUlBGJZhsfU
I0Dc1cb6vJYWZqpyLUqPRd71QZcJMi0ENZacdCi81tNEu3p9H3gPM0v6v9XEyu0V
KyraIaNDnZ46clzjaqYTbgL1y/mFKsN58ZQYSv+JnoKE+K80BuPme4CxhzAqTz4t
oAIdTqJxd3Mj/suzdfVf2XiewfUe6qm4ETBw/Job9mgBNADbKGNsses2WhVWupDo
HJXsQGff8DrvEGEtdqlw+ptn/acwlOeerixH1+e5ORT2CdfnUAqva5Edtf1m0tDp
WVCh8EAo4wdyPUydvI8CAqsuaOIAtVpnPDd+kJO+/4RKKBuoC0ztJWRbrWDA9yFo
7R7fLC07G2xgdu7p4XZwtMC4xrW/Si9bKqUho13uBer+WHe6XZLcC1N8Hkxd24yX
GANnyTMGj58emWVmXmOrbznHblfEFQvMilbRRXelr9I/Q494uxTMDR/G7vVcyS5Z
okxeQf5OognGAcVXVQPck0O0MsXwFBws9CcrSWdNBg1Y/nvjUaj6Xh8BPKOORFKg
R7Efgpw9Z0yHhEStDO5qDpsXmLA8LZfO7SfvPA7P3dmROne8T73mfjRJ7WO3HMqM
tNPyANv3ygy1cEC5JPrB8+CFMj5GCEU24pSND0kU2TZ5RMCYwKNGjiezgHrbxflC
RIOPNMwoGjIKLGdUn0kLiZ5ZSTBp9Fv87sNymWUo9Hr/vbxYHdx849IrDf8X0BAe
srltRGwySBm+mr6G1dl1d1JB1anDSYTJFWO1ahE+9rqtYI5lHqIA8jcOvspfLcU4
Rg3kgMqI1cZszXEsduV3jMp0Fa06q3bvE2Jp4c2px4tTAD1yaaKGqXOXN3iZbGOs
ko76XsdgYIfYNpMAVapWfTymFpe9l2KGtrrnET1VKcR35a5JtJj5IwcGcK79afLn
iT3V6eZV3t/77RID56whNjf8VVfakvigRtKBmEm/FHfhCaWY06LpJPcwzGFIm0sL
F/Ig0KWYhCe/VPBxtbR6LeiExlM+yr0dI4VQzbOgVByxBmxd+NwF1dfXlvjFS+Bb
QhJnAmOZOWvhtmBoaDdpn1MhZJJNnygM/hu9+O3dgXuUZb4X9h9aOZAb//Ov9Rym
r+126Hc2fvUAWAcmrdSP8y9+iYJutKfGDr1zJy9CVjxpQd7Yl0prvGgk91o5Tf2e
0xWYQtJrul3Z1DR/T/9EQ0Hmsud+K+zHwJHuL/eHPSJzikDIII6gKCmcpe6iWfXe
8djEejo+BfvnYWNg0lo8ZAK+UVvnsnzcIr8+BTf+Q0ugwEKOTXH9J+PkX+2o/I8n
At2e2l3LxC5NGX9QvQNmPw/3n5y4Y/etqxEJBA8H6Gvz4lqyaWk+hTyH3aBRvBt0
pvEf4y/lec27PP662eo4Y7I92DPkBYXcP4kupIikv3DEFM4hAKn4WQqt8R8sHhvL
irOaIp94n/ic6sN/UFLcufksxat5Ot9PkweHZ/Z5UWPIJBxWsY1yIb5WWn1gH5Ps
X+JPcimXa8XRVW0vRkHJ8Akzvqp7Ukxt0KC6WlD7T9njiLlwTvUAS91+WACh9Jxy
dcTWvM8yw0RkB+8Yuzf5S/4EfscmtsjYMn9eXVuyCvPf+XJSKUs5S4En/zcrmtg9
LHcTZFB/0OsxKJjgtwJK64WS0uZOKy/Wx7M5DtRL9R2Ibxpzs9wL9tdCxsrYnYxS
kHB5OWqBbt8aLBjfCfLGRnAhKGKa5LE1j8/eK3Ydvx1LxmdCQzn5sp/gptVXOzU3
cI3pUzd57ZlNozN0RzWC+yNznRaF5mrCUI0kjabRnf3f3JKXVdZACvWgH+99H7hp
SgK21Fu7F/tN8kMOMgaX/FIPzUX5yTwWc1DAOTvmHdh4yUpTKRq9iCyHKbEqzMnX
sfTTMU7+5EjP7Y3oz0ywMq9d4gIC6NRCKBWAhWNBVWTu/aMsEsUe+pYbFP+QXzKb
8zbfV+EGQfYYfrzC6vZz00pMMujKZP0Dbe2oh1EHbeAdhLkGNuN4ExwL4h5G0kaD
ZEKTyJfyKu8CqrVSgMFubJI6Gz1ecj7o/WNuLFq9wicg4dN/UlfiwtsDbaMKB1jM
E/nyz065346rBKL0kA4CVJxiVQjRs7uDPGujq2eHCxw5okkz5fygBTju5zhd1Wjr
8vpsmso6F7lwHDlJbBu5prcLVj4hCGgm0GmZwlgKr8Jux4SR+TtHR5/l1rlU+tO3
Iwe7uAgaV6XwaTAqGc+OBYslnMdyRcGYhobbbtKsvA1kJrZLbiF9xV/4UH+LlYib
13BEhOamttVJXs7EGKYjuXaKIBdthNPEomzrOqzQ/zWXnxCW8ofAPWkeByypUydB
v9SKuDEgC0/A927/qypv+2REQF5oFc+F4IxyQsBNbkgtDghA4LPLZByXMGSJzts8
4CkJT8KhVpT++OaNah+uTFX2ougUVk6RQ1+gmk8Z0yBD4aK+61f4LSZ+rDiT846H
rC+/6CDcw3P2Zn19pUhbV28Sv0LWCLSwJl+uI9L5Q9Wo0K/HhzzAuX57lbTKByYr
3CiGXue+Sx1aYRnZFYgg+B9M4a1r7TQR+XplVoC9RJ3iUkIkJwkUr1QjrCgpZjOT
UASWqaCtW/6LFsA1LVMH+eY1813oioQQQTP4VU6F5uGZcZm+Wo41DQaPW6yDodFG
mv7p/OReqORyFVqCoSnAtVPBvwwDSky+2AuilZF0xtvJ9ZWV7SUEqqnB2rs4GkUN
btCAC+UDUgib/xu2KuG69ZDS9ZLomH06jEai4VdBVP+0nL5AYROwWvT+15rhMfs/
cICMFnmeE1hYpS8LwiKASySsAXEYXQeQtlTqcptWs5IJveXYXOLJP2hAoNpCM9mY
/5hntlOd4L4PyPb1yhElVpE3x06corb19Sp1nUk9RMQ5M0TqYahwUg3n5VywvfLD
OQ8DqH6uiA+cvZKmRguNYncTLfDDDMn3/M7vkbDdpbxoYdQLaWuKLJtabAmCdfpi
KGp6r5HtKLHKoPOLHZZxPkYj725Ka44Us+1dMnpwGeOxedPOIVAItJqv+d+jkYJZ
a4Io9RNcaEumdQy38kN5IGNP5uvLfYOQHJsb5+S5hFwXNj1lQoBoJgLTV6g7JWy5
mMh7A61NM+IJSf1kmmW2VDGNY5S6h2AUwAZiXiT39tTrhzYoEIj43wPlrOOMCH9h
rWIwqyXsLjZLcxGFE5x/VZgxocqo6163z9iIeTAwO5j+wj2PjLb7tT6EXmlRuc08
J18Q6eFWqIbn62mSo6q+xJZQS/URSBEBWGihPF+QSb6+uWfv2ABzYVDD0L7vLJMS
Q1O2WScjskhc1ArLa7jdYNyMYYJVT/ukuUb58oj95cdRrTLfBaRjMDiEb216aog5
4tXfpnAYwKikcX6XkXezWBkvwipP2e+zeTuusjylgJGI2tOd6h36RoG7fUMzv0k9
i3rIx0vhnUGg5UaG3N7JajnuxtXI/AR8Wgov8qIRfNQrSdE/7LosSt71J2zO/42n
nMuBZ3aLcE2AmEMueWlAf+Bbp5Yv7JC+J5q7DhJjExjcR/BoArt8XEbFPK9I6CK8
laPtx9PN9bu/CygPShd262kiPajRbVeUPRQLkLY68NLJU1aVJ5uupEW/eJx6Jhrm
SBplsytkfxYw7NQ3sfoF1HxeIMsKgpfFtOZ/w1d5Lv9n11C5TRW9BPqwPtXl15m8
u7Jv2rCIvvM9JvrBHFu6sxWVL/wdUqbB5+0RqsjOOxvIWazjEJpkR3nSGydISc16
/qtp7DMKyfO7RXB4rUaOXofQ5VA4UzwfsSUC+lKAcQnRh0hrW2fb04xyJaU7w07H
ufbrvMyCG/vW9U114diJvLVDBR055aH+H8mDKy9sKv29m9OjXzN+C53BqrLjBVDD
wI57WEqHA6RkLLZxL0x0+TXB0CGe0bHXYHb2j8vAI23NKazTVZR/+lZ/n1N/8m3/
kab/U3Iv3B6g+nAlaonc1LvZ6DDcM9oi4c49oDCn74NkrfCkKU648twr/lgceNan
L/B2SuYv5RPNE1Tro16GUXn5dE6c2Lxw2ChFlKyIgs47V0NXPeT0fxvrhi28fWxH
fLZn8Tq1T4a7/MEHcVeOnwuB+CkD0jaY+OI4fDfhAxIXA6Wf+IZGrpkZy+ZmSemK
Z6YNdsar81YsinnqqV4qCv9vr+GHX5ERiZaJFQRuHjKBjg+COm1lPxho4KsfDofs
kX/EGHVdS32uLcBoLm+8J4588ipD0DAoNEN+oAixhDa4hLBVnnsTPi4y9LlJuMJI
el4r9LAdQduHCd3XjgmBwI4yg2zSEuRcsSTz4PqRsk1r+uZ1/vflgYM+lr/70wDd
fwbBRlrX7SGxd2sJYIcimj5tlfTZnAsCaXm9SlBKhHFftwFsCYxN7Hs3Xj6mSNDP
z2Ob+vV4K2SfLnalVCjdvvMoJOzFcoI//tgv74mSBkJdZVfrvMl8u8H8D71bleRX
Kcn+sIkXJMySnYjnE+4ualvpisA+nojWoREtyez678FU4yTiaesUywrG2srumGDB
NIKXocDOnEABmLkgqxmAXFBOM7nO2LrUuOQZOx1RXK9R35q7zURAf+MD6QLmBToN
N6+2ekKx7bHEtFWmebG0BfrJBHJ+Y2SNiClkoIxPXZcAog+zHcrCtGGEMGY1Rdg9
4dXOSLSFDNDnoMx4QWt6gJThFNtQnW85lZjYPFPgX5xgoXgdynA8tdl5R8EJ5Wcl
A4uAy3jAAa+8FByANaT9YioPsm7kuwuyd7E7ssU8AvhT0a/cwmtG6twAM2K4IE5l
XIc/H36NLzrR9b5FHpPdesz+nA5eQHYpZDEu8ZD25w2+Uycr/04n0RHtF9rbO/fI
v3iTWvv+OdlBftuCEfwm6OrvVc2+cGdih310UiXta1M+JwWBg0u/Ci1khwQAGC/g
W6BAQmWNUX834U40VYOMWWB+S6iYdy9VnBnAtB2xMtp52ceG1cwb3IqjJGWywjxy
3w0ODDowUXMJl+qeDjg0v++TuZY2MBaAs76/oxTHHWRxNmRmZYFby3TKneWRHy0c
1P8IMwswGugyDcdKvZLy2bvnSZaOPRSIcpNhAACGny3X3Ntc4/ld2leQsvsR+XMG
JeSQH/bJynq05Gkijs1Ins5YuglCuhL2VHlYjm8DQxzJyGzgGGHxVzpk94lrH5Po
6MlQSflwXL5chv8jqbqL94ues411U06g/W0RS/1IV1Dp3O4JsmmrLxmbpe8NpFAH
3JID04IkB2MkJ7xBsNY4FJvPCMRHbCNOriBecfzvnX9CCMcOydvzlE/8BLwT4tSf
/FQFQzU7ZxKYFkSG8n0KvOe8xvGXkcCdMDrQ8N+oxbfl34rOdsV6djR0FD2+12p+
3BfzASwsY9zG/hLbnjkk2eujqdWM9lK+bs5BOHS6WwfsGfn98LeRlCcA+ReX37EN
FHUNbXm/dE1o7FD80gAANXrk+dVQy/rLz/naP74WsPSF58cwsDLXh/EaNRLZ5fD5
VeEzvlCMERL1viJjO+x2XTfSr+kmDwTpNTHPvmdz48FFFs5L6h4UcICecFNpYZNt
xAa7ZY3+OFiO5bCFrbqZc2VYeBzJ8IVy1PJKi0yyvRhxVbSNmepgy2KfBXEhOsRJ
ZJSHB+bkjWu7n9rK9K3J49wboPig44fcbnTldf3f89QAAC+RWy4KUqyRxcOzoNZX
Gg8ofcLGAdh4A4/M1YdcIZzIDOgPzirJOZnhR0uzm8cXljQQW8VM0lDdkWRabfxh
kANW88vc0VGIsRvQ6cifq7c1X5frzdNQrfW34udzqjFUiEddW9Ga2AVOAwEvdVAS
/GAJUkTgrcB9icf9QMhGmf2SciBFNtVfXN7SqOi18NROuGy69T6znIAAa0F5ANRz
JKS31oWQb1M5V2TxliJpNUzinOCy582oK9AN23bz2Nw9l/a9GRH4J9n96IbIt8KD
0Dyz2ElujDDKNgp2vObeAtp79Ft7fvABIoLSBQizNSV5rJEM/21GUCOZ9C1vOpRT
+IxESSA9ti3nYt8SlXbgpiDAeX6bcgEXz6wrAHzVL5No4GD9o+WCMxN1Swz11YC0
5277aEUWqtKg1b3s12/R0EJ3milURHyqy5EIe5Of31DXLPD++vO6dbZTQwABKj36
PHJSEWAW6exwGL/M86GVLqXXSNaWqBUgZRIvtsXe/llrg+UdODjnz/jxN5vQ2VTy
cjxMW+xf1nde9fbyyfMuq8nkFTr4p59De/GFNfjenMVRyJTMbuYM/fFcXppWTFYO
4R1NdT5veUNHsm2X6TdByYqtg+m4BF6YXW75mnn4d2yiLOTGoTmfLOQh+4JiCkww
akIDCjAf8ZPv3XjStq30xX6pq3aGYzsbgXYfmy82jX/cObCCT0kmKVvnsAE+23ko
dwoCYmrLc3+Qdpi357nGL2jlka1Krb1wjxamJHCaK+t/yrh3o/gAAbTdW+CRDk4R
8u6WCGZnUHrF6oW/5bAZ9+MWNEbhxsUlYOLb1fWmuKBwar/qhYKRXedFk6dL/bwp
TkYWmtydH6gGhP6MpJBXMZ6iK16lVwnYtxjUk5BneYGwSlaz3jWRlU5okllHXhvc
5Od0smy84AbfCgLtsU1dWNXi2t09aNvKyFasOyoqk9hWhdoNo98Pi8o0jt6lrDFF
VDJH4xwMHxWtarzvjBQJW6ggBzENstHYl9OOhFP7OrmYUWYybfi1WOGMIpIZURea
CV6PsUEffJje4wPGZlx58tDVS6QtKLDCsUt+qEzATHN9GUlb4bgdAuxUEHEXYAJh
ujJGTW5G2e1Mvl3oe8uimVKYvJfOCMGr65yMYkQSis+4EwAWp13y5u20oSTApg2P
cAkrtgzstfCPHsgCg3aue3VnLX0cwtRdsOsmEMIcbZuBN03kA4brLZp8YfuTmAFy
8agOr6O2CaT2NQFUVLDuzHydmA5F6jXO1AlV4Gsk810s8gU7o9bXoopK4qHv7P2b
6jzf2fs31Hm/s/ZvlwFc6juY0VTznslItHJmyes56CNfH//ZjYzcgJJeu/D1KcJZ
/FBIDHd0IEoRY/8jDpSmaIyaepD7p5/ZeY3/m7fe4d/71w8LfEvWXRlfFQ9cDDnE
vaJAzCrm60Xfy0TpT9IHaWBi27HEagRil5aAW22Jmqu2TU50OfH3SdFNYCtybNCA
pRqIbKMW5q1Bka83lSE4hcMKWf8x2v6BYtw3jYj1Z57pKVlasWRekAt3X4thYfJG
7UIeLM1CKqy1THtw25zDyfZ8DLUrdGzHIuXy33CwIIGqY/TDCORiLpGGndLgb6zl
uNo9Ls6AWbPTGU2NgK5L+tRshydYf+S9nc1S2aD9nQQJqyPb1TQ9Ce4HYO8y6LZl
edsPU/jJ8PuoEZQ7n5dvMLLlaiN/5ThkT4QdQXKSSRMe3cmxpV68eOE1KpHgpVUS
V0l1EgjlipwEyUCp0JqBsHXaXFEd+ohefr6Ll1QxO6zjmMuw1aDWfd9gt2onBN8G
sJJPnpqxzaFkcrwHu3J6Zkbz5Q8Piav55Hg7qzC7w8poi5YbHX/dK4sT3/oCek0K
iCPNxMF9CB63vCtmA7I3oAAAWE1QIAEEAAA8eDp4bXBtZXRhIHhtbG5zOng9ImFk
b2JlOm5zOm1ldGEvIiB4OnhtcHRrPSJYTVAgQ29yZSA2LjAuMCI+CiAgIDxyZGY6
UkRGIHhtbG5zOnJkZj0iaHR0cDovL3d3dy53My5vcmcvMTk5OS8wMi8yMi1yZGYt
c3ludGF4LW5zIyI+CiAgICAgIDxyZGY6RGVzY3JpcHRpb24gcmRmOmFib3V0PSIi
CiAgICAgICAgICAgIHhtbG5zOnRpZmY9Imh0dHA6Ly9ucy5hZG9iZS5jb20vdGlm
Zi8xLjAvIgogICAgICAgICAgICB4bWxuczpleGlmPSJodHRwOi8vbnMuYWRvYmUu
Y29tL2V4aWYvMS4wLyIKICAgICAgICAgICAgeG1sbnM6eG1wPSJodHRwOi8vbnMu
YWRvYmUuY29tL3hhcC8xLjAvIj4KICAgICAgICAgPHRpZmY6WVJlc29sdXRpb24+
MTQ0MDAwMC8xMDAwMDwvdGlmZjpZUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6
UmVzb2x1dGlvblVuaXQ+MjwvdGlmZjpSZXNvbHV0aW9uVW5pdD4KICAgICAgICAg
PHRpZmY6WFJlc29sdXRpb24+MTQ0MDAwMC8xMDAwMDwvdGlmZjpYUmVzb2x1dGlv
bj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlmZjpPcmllbnRhdGlv
bj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEzNDA8L2V4aWY6UGl4
ZWxYRGltZW5zaW9uPgogICAgICAgICA8ZXhpZjpVc2VyQ29tbWVudD4KICAgICAg
ICAgICAgPHJkZjpBbHQ+CiAgICAgICAgICAgICAgIDxyZGY6bGkgeG1sOmxhbmc9
IngtZGVmYXVsdCI+U2NyZWVuc2hvdDwvcmRmOmxpPgogICAgICAgICAgICA8L3Jk
ZjpBbHQ+CiAgICAgICAgIDwvZXhpZjpVc2VyQ29tbWVudD4KICAgICAgICAgPGV4
aWY6UGl4ZWxZRGltZW5zaW9uPjQxNDwvZXhpZjpQaXhlbFlEaW1lbnNpb24+CiAg
ICAgICAgIDx4bXA6TWV0YWRhdGFEYXRlPjIwMjItMTAtMTFUMDk6Mjk6NTcrMDE6
MDA8L3htcDpNZXRhZGF0YURhdGU+CiAgICAgICAgIDx4bXA6Q3JlYXRvclRvb2w+
UGl4ZWxtYXRvciBQcm8gMi40Ljc8L3htcDpDcmVhdG9yVG9vbD4KICAgICAgPC9y
ZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1ldGE+CgA=
" />

Each `immediate` action is executed in sequence if its dependent inputs are available, allowing us to chain a series of actions to achieve some basic addition and multiplication like so:

<img src="data:image/webp;base64,
...
OnhtcG1ldGE+Cg==
" />

<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRjAjAABXRUJQVlA4WAoAAAA0AAAAYwAAYwAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBIfQQAAAGgRW2bIUn6IyIjI9W2Z23btnds27Zt2/aV
bdu2PTtqVVbGRXcp89/7iJgACCd/Y8btp9vKWAAAaqnThatfJpRSIXSdazGxlMZG
a1zXhUYJJRCkVWbb09sz3uDgTvb7WSmlzO0XBwD/3JNSns0RKqcAAIoZl5AYZzIA
AKqqQnBOiovrlyullGf/YO5I2+AUkfdKElBHO1JKWR5Afe2fFoNnL129fuPG9auX
zh7c/J/XdADKFN1gAEBK3pNFnQ1p7vjML4vfxUCfJIt2aLbqsc/2OzJIx2/77q9s
+jwBANXgwHbJ4v2fuqOCDNwhlpe8WSSCvqODfnpeAx7XXgau4I4f/YHsD6JLnY+Q
lDJ/59Dfoz6wA/l/cMezxwPJbVtv+yMnpX1n+w4Z+Piz7uCdCgN5sbATdwe8vMdL
e14Gl6q9vdRbdQmzrD3e2WtZzA1EMVmJeZ5xZmVTk5OIUSHgm3U+z0i55TcuNBop
TZBqlxzp5YvVuG6RyHAmWhdIj+fVUrhKIkA1ZnX+T3re1yWBWzR8QojBDyWCuf3j
dY2ESxF8WKFEMa8LNdQwMVNr+0QiaddTTRYWZpFK1yWa+TVVi4WBaOy9KxLRCz8r
GgmN87TrEtVtWZyHxLS4mQ4ucqwQLBRTq31fIltQn5khCJ51SqJ7/T0ugiKmMkXi
68yKNkkwgv7lQ0jeKslEEExL3y1RXmUKFkgRTXNxckqqSiA9e4NE+nSMEUDhFZ9g
JRuoSnFRdJVEe2diVDFMf8OP193SOitikAUSb2eMaQAA0Z/NRUweflEnAFxpIjH3
/6lwAG4sRU1OZhyIeOsYblc0lVBR+j5uzueCUr2zg5vsrxKaPEcif0BQLXsvdo/S
hXjxPnZ5XwvjK4l9YS3dqoOe3csw+6HnTNOsWejJ5XrCavx2xqbswe9Ycvpx/M6n
pZ/F70pa6v9BSuIJ/M7HR+3D74RprcVvlxE1F79VhjUIPWemrjdCzx4g9J/QK2wg
6FsPscv7jZLnDmP35FlCUxdjd0ajxOrtIDdGJcCrPETuNwWAf3Qat4c6ByApq3Fb
qBMAMDqh5q9kAACwd1E7+x4rArGbEXOmZ0Cx6q82Xg9r8eIgdideh5+HgEp9B63e
SiD2xl6sbuawQBDVLR8np7mAIMmrF3Dak0GCAdHCxuhpbR2CT1yLkLPkeQiRfXYF
n4e/K6GA0cmHzkANQibPr3BwcdbEkNCAfXcdl8NfMAgn/ycPkzu1BISXt83Hwx6s
Q5hJ2ug8LAqmxJNwAX1hDhK+SZkUwk9ilqLgX5hJIJIkdV6e9wqXPE8gsvS5cYVe
ezw2g0KkSXK7Am/91zWRgAtZhXteyq3HwZ3m6Fzv5A3Twa1KmeOOR06UUcDFyUNt
L9iDUsHlbx2x3WYfegPcr9Y95HOTfbihBp5MKbfJ57jEt7VCMng3q8lx24mU4zvQ
MBU8zr8fsvF8bvjyz28e8r0BCNKMb+uPWnvVDs1/df3Iel+nE8CSmok5H1frt2Dr
qbt5tj/37tltC/pX+zgnyaTgTgBWUDgg3A4AANA7AJ0BKmQAZAA+MRSIQqIhIRXs
BzggAwS2BtgCuCjVfvot96/Ib2SrB/Xvw3+U3Nbl37NP239w/ID58/431LeYH+ov
649ZDzAfs7+w/u/f7j9gPcx6An8p/rn/47D/0HP289Mj9t/hF/b/9vvaA/9/WAcA
12q/4H8hvQfxqebfZ3+jekr40OofEt9sPxP9r/bT2V78+AF+P/zf/Cfl3+SnIOgA
/M/5x/sfuO9Mr/F9CfsJ/rfcA/k39I/3Hqt/vvEX9H9gT+f/3P/nf4X2Cf/D/Qee
76U/aX4Bf5n/YP+j/gf3e+MD2KfuT7G/7AJ43H32bIPfV1ZQSdoGU92JRmfrtz+q
0puw85yx4qrz5TsSH5x912iBlHEY+xfneluU0vwSjMnGHRFgXwXXYALK8k9UMC5v
Wgr3TXDuJ7Stmo66PVjwbXmazL4L2TWOJvmrUqbu/3lTMEQEQLcHIojoMNcsnqDr
NMUm0fB9BaGyW1UAEoFmetzNftAcjTSswDVBe+1AAYqXOvgdQfzURAIWh1deoR6s
D7Xl/6FHNcR17zk1W54Zf7baf4NzJhd3qE9kSR5xWLTV8YcEbYArguPEViwOZ6Eh
ImDMY3FiSMRgvOtzxbug5cSYEsc2xucIQonf2IZfQBg9wgAA/v/oN2X8R59hrAX6
uBisLe/1gGv6JofL4j24b6pvEW79wOzdnebLYvFoZuatF8/dsZrJp9SfhqixZR2S
1POFQA99w8/eTy6bL0ppRLrRJkEdoklX1ehWjO2Sf69mDb3T/zMJCT5skZaLyrRf
b2fqX4Kv/e2EjerKV+xCdNLVtdU+ktKmUbhaTZrRLrxj+ukW3jkRO7VnQI0ciodk
8ZI9lUBUBoQdHob9TwRJn5DXFPcJxUi/iD1pDebSNf3Pz2/EhmWblMHLCP+Zn0LW
Q+F9oupaUV2zCU8J5GU9KRmQg9eEvdU1HXcL+IHck07Mf5YfBx02EXY5ZD/H9X3Z
ZcI358QI7zmma7wznB7SKuc9RCbi6SY/FLR69yxvKs69FTSRdoa40llqDaEuYs7y
ge0rLb4TFAygGVcDjXJqpaqYfXqSCbkd1RrZii+2dRv1UnaPAfqhEKpzAJaUIFL7
A6CgSItms+Q9iPkPjBRHTnduBcAR5OJI7Vd6eYx3GpX3vXOlSK4KcwFvLJWeG+ta
2nbdD7FnKJOu/9q9aU0UbKo0nOST7/fwT4lPxnKyFEr+XiFoK2XH/FtuYHmh1qtL
+b6gqfZbeoE8wMmnClofgnqxJc812yKyRaQF236q+a86h1TlAwifFff7NauX0fo/
07cNW/g9oaJAHKyHxKa21pLsw/HZqKv55N7DeqhFnNxWO5j8SB6PanfmZMeK/CKE
DyXqB9xbvUlNfJGC8w/lN0gP0aoqGC4Z4ty2vm765aYaiZuMCQVRJt/KnVPxDL79
w0enghM/6dsaCYd6QS4WiAMbYyVZtVTWA7MRoSU0iapfvwILJOAp/CN/N97y/7fA
mCiYW1o9tFnDZw15OsQw3YEkHLFD6F8Tp2d4+tD2iIfhBfLWzs/psgs1C2KVqEVu
M/+j4VBAyi6MMJzFjBSkuehPMz7BbfwDMY03xT4ae4ejNRCGB+KSaiN3Rk750NiW
V1n7s6ZxfCwlmHfAFTf86Y9IgE2JEVmg4Iox6c9+eJLwkMCvcrI1nqnTItHO9f4P
rEsWG/hVeRx4eSanSx3HSqPHH1NZlPuJDvBe+ilXBKggPrK9yU0aAS/rwO+GKmTJ
9mIeZlUIJ4Af7bqLdfG1MmBZvI3AD3TorjAzmZ8aeWKSTALuWCfl1TItVsulOFAa
xT1Kc70TZAqc1Ip1xlXUzQtCbd+4BQssGbz0lQnfpj4rK2tE/yjyqdMfAeWU12ik
flaoM9St/cshxMy9W1bc9Ygi8g+/pMIPBgS1I4FbMAmKB7qvBTgX8Frt3fWfe7iN
L/7bfykEXCD3qIhNoplnPGeqid4idf8tRkI5UFTdDrJgjaHsSkhjhSTjzjhYcNZ/
pDGvV3iSTFe+5Kcb7C/fFkR3YpRBtQT7IirMlLglMinwGTBEP54woseL2FEQfhjV
cbVXTMRP08A1ULLqOewo5qooICvj0///w/jl5/z3TppNaMwSXmKsX/CpWz4d/SEO
wIOBU7+1LEzlRmIRzuHkGtBy6rh8n91Kov8ilcEdCGVgcaZdPkLZSFxR10RB60Cg
x4ntcQ3I4ojyxpKe8kCY8m/fcBcoPvGSCheGmf5fvUmC6LxC/61Gna7iFJwuwMRl
bTuSYb6sVm5V806xp0GUKXy29uEyrdTjXpPiA4FtleHXVZb7V1jb16Gt3EBuGgb6
RzX8f8d6LThACaJqv7nAeyJ1WRnAYJ2CBCuR0MlGuQWd3cg0MDL+f+H7griBqGe2
HapROc1TpIIecVsaidl+/m/w/kW3eLHIMjJ5cbF65iSIFqJkQJrH5fn0QEa9IVpW
ZVznDxaEoL6NvFYriWTTum4la/R9Facts0BjeDotKCI/nWu6P17bCmNPnWdiQYIL
gp7jPUlakpv5QaT+w3Y+W7hfC2t/jLnCO87QDhubrasFsZLC+DiLgFS2n/+6NLYh
tB/dYGYAZVOgQWkWO+L4Cny47XdPEF5z9q+OmMGwXoxNQxePR1+j4FJQvWeA5OJU
JEZmCJUtUfIUwfHqEBT8y4b32LchlDnGbHdXaZ7BCR/QeRpMY5Uf5+Xgbfva3oWS
CoC1hvvk8fnnn/59OC3oWMhEh0B2J2SVXsc/IxubBBhT2qO4UOEGZAVqhzdHLm9F
7Q8aNM1e8uFTsFZmr2WCfiZb0f4951fFqxcAKAoS16HK1a/8WfHBP3mHGqtcRPmh
K3/xir4rd9CRqdBuu0oiciD/mdl412Jk2CsWm2aFuoDxsx8kvwYAaoetI/BlxZc0
SVsoIB0sDnuUuW6IrkKZnXgC6rTjsvu9DjJ9LTES+s5AmjK5d4mestcoWOUDYhxv
ZGvcgkCfDTTu8pVFiRyDAQjWp++zNV8EkzMPi6aJe9OHVJUrgUSgNgwE5nlxCaGv
lBobUYrHOhSptiLQ/vDKm+AlSm3G4j8GQBF+/+pTFgwIJL6hSWu/nf/dAIWSAOR6
C5RpFdOsltnNcrW1qWM9gJa9aLcVdI6vjzTb+RNLMQ0SAzssFGPeq+R7Fzcub0vt
bd7m2Fl9HHnIDotd4e/4ZP9DrpHJ+o6e80R/NnMw3nrBE+5MMHl3HPUyK4bcZJev
/LHCc5mh+Oo5HL2COv3zXykbg2q9FP+Y1dBjH0x9YV5o/7jV3vnl9Xz4jjbTCIIS
4Ss2anEFQckyc+LjQM2liXWTRSDytwHOA5cS6rWz3dkdpy+u2tst4nAuPlhXQpPs
0UujsixQ/absVYL9440fRxO2oVBvIAcLHzjsqnBx9DUqmPtfapaosaOmO1UVnt4P
XGiRnd4/mqMnOJ9kZM/4CneOG4i2wgbJAkURGDiyzbPPSa+E9YDFDuDKI0Wp/d1C
x/clfOHNNA/phW1bQKhDtmLDXn7wpdCS92n1AJKA7sCWluwPTXgJB7i1Q1kvqmmM
Cu0KGVLoTZwgRQYW2Ne8nFFFJtbH/X4cK/kjjQxqsdnWLIzko4L1iMr06E9DyuYf
d9W/6XAPBnsv62NhGUuQ6WfxCctD65xfXERD8hxJvHKaAa7/lM6W4/y+DD7E/n+a
XPGJgiOKP6O1fHxggrey+DqDkPUvWrgZ3LFfV1Zf7v/7FnMvnNy9S2eklrOe/rJ+
0/o9VyeqMT7p/NRm7yMdS0Q8oczaPRAfPjzkX4Op9uAu/2n8XuLhVJW/MaxgWkzp
T7KJBNOYgMGxU5ikiITE7KQqdmOcoUxAmAURV2r4QHcRVO9DxefWl+vv59Gfyjwk
1vYvNnbSwoulILAnp7pykZlQOgOnu3Lo5ySpjAcPFDGbZJm6q32peXWabycDbgCH
Gf4Owp5EhBWTJJ0z6hiW8fIbUlML3IAgEhzERx/jZiF7jF+F2fU7ZfZrBlYz1HLD
mNOOWryrenAeoLjvpyk380G9MifioUwG8wu+oLL4lwrBp3bkUjqDJweBJx6XElCh
YXbU8RNduQ7M6+3BRD2WgvPVeFs+vL7JhGihKZg9t1987iF+5I7GPbeTUqIUqEIP
RMHdaZhJE42leiRhVs+xm15o4MX1MnzXy2UTSdbjt+S18sUF8PiXpG/aFJrTsnyP
8B4IJjwujaCw3NhUxvUPlUP1ZvOOQHpXrSl/pHAd8ZLsWwswJnc5mJTmI4JzgZ9U
/4E772jFeA56o1iWE3osT//2yLg1eK10GAxRR5a0AoA0jCFtumufv+b5ZuSR6yaI
YSUbbcMrtmZXYUOBpgBGQ47U9wXLeAxT8QpBkoEE7WFBdxeFNRNucYuFpehKv1QR
Ht0o8RyTjiNCLL58GIZUV/du4cKXtoTDMvxXfHZPycDaPTA4jq0779ELL7gAVOig
IKnHEdfYazkwOwXE0u1N6076k7YfERWeASgiQ1mtfrt7KGzE9nEmftVcgRsdEGZY
PgwV1yS6VU1oa5LDtMpYoYwl1uX0kTj7IBM7U83zk+Sb/3o3Di/D3Xws3Of8SlrJ
sn/5xh5ARZPXu0TNbZEkCVp9C0Jxgr70Iy9KH3hRo0DiTa98rXVIpIYzfREvhwVC
VBMDW41FPScjqw7GHO31G5vQCss56385Dm6Zdyz7IRZlVbEge//ggYe/mYCXxjHw
6kK8hSl8W/HSgWMGxtk5qLdXl/6mLPanzbql4LYwT6Jb6fsvNMWPDXWZdpCVkdJY
UjhM5PFEDZtE1ApUYX+0waQlrpm2A7JQoiDjSs1MrvxcKJ35eM+Q+aWnISlL/KV6
E2pH/wDDwWHq6cnA7rSEE1eNVMwoQHdN6etMlbYWYha+VVqFEGznw9DPE5eBYgAA
PdvtrrXX0CbwtqeNjHoXgIuy7nFG2G9NlLxHIubHjxxzzeZge0HF2uF9Nw6+RBTb
uDQGAIw4Qa3jXxVdnhNBmAECrbUFfo7xHN2lWxhWjzBbhCsmfKsti3G6Sm9/7ik/
/4UqJK5VbgpHViAObOe56CxjuKSnlfZdZ3dz1mCaLDE3ottwAAAAAFhNUCBYAwAA
PHg6eG1wbWV0YSB4bWxuczp4PSJhZG9iZTpuczptZXRhLyIgeDp4bXB0az0iWE1Q
IENvcmUgNi4wLjAiPgogICA8cmRmOlJERiB4bWxuczpyZGY9Imh0dHA6Ly93d3cu
dzMub3JnLzE5OTkvMDIvMjItcmRmLXN5bnRheC1ucyMiPgogICAgICA8cmRmOkRl
c2NyaXB0aW9uIHJkZjphYm91dD0iIgogICAgICAgICAgICB4bWxuczpleGlmPSJo
dHRwOi8vbnMuYWRvYmUuY29tL2V4aWYvMS4wLyIKICAgICAgICAgICAgeG1sbnM6
dGlmZj0iaHR0cDovL25zLmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAg
IHhtbG5zOnhtcD0iaHR0cDovL25zLmFkb2JlLmNvbS94YXAvMS4wLyI+CiAgICAg
ICAgIDxleGlmOlBpeGVsWURpbWVuc2lvbj4xMDA8L2V4aWY6UGl4ZWxZRGltZW5z
aW9uPgogICAgICAgICA8ZXhpZjpQaXhlbFhEaW1lbnNpb24+MTAwPC9leGlmOlBp
eGVsWERpbWVuc2lvbj4KICAgICAgICAgPHRpZmY6UmVzb2x1dGlvblVuaXQ+Mjwv
dGlmZjpSZXNvbHV0aW9uVW5pdD4KICAgICAgICAgPHRpZmY6WFJlc29sdXRpb24+
OTYwMDAwLzEwMDAwPC90aWZmOlhSZXNvbHV0aW9uPgogICAgICAgICA8dGlmZjpP
cmllbnRhdGlvbj4xPC90aWZmOk9yaWVudGF0aW9uPgogICAgICAgICA8dGlmZjpZ
UmVzb2x1dGlvbj45NjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAgICAg
ICAgIDx4bXA6TWV0YWRhdGFEYXRlPjIwMjItMDktMjZUMTU6MTM6MjgrMTA6MDA8
L3htcDpNZXRhZGF0YURhdGU+CiAgICAgICAgIDx4bXA6Q3JlYXRvclRvb2w+UGl4
ZWxtYXRvciBQcm8gMi40Ljc8L3htcDpDcmVhdG9yVG9vbD4KICAgICAgPC9yZGY6
RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1ldGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">⚔ Runnable Code</h3><p>Much like training droids, these runnable cells are designed to aid in the learning process.</p>
</td>
</tr>
</table>

The above could be represented in code like so:

In [10]:
# ⚔️ Runnable Code
a = 1
b = 2
c = a + b
d = c * b

## Domain Setup

Let's get a feel for this against a real domain. If we don't already have one started we can create one. We can then make sure its up and running and get a client to the domain. More details about this process are discussed in the previous session.

In [12]:
# ⚔️ Runnable Code
!hagrid launch test_domain domain to docker:8081 --tag=latest --tail=false --dev --silent


⠋ Checking for Docker Service     
✅ Docker service is running
✅ Git 2.37.3
✅ Docker 20.10.17
✅ Docker Compose 2.7.0

Launching a PyGrid Domain node on port 8081!

  - TYPE: domain
  - NAME: test_domain
  - PORT: 8081
  - DOCKER COMPOSE: v2.7.0



Current Working Directory:  /Users/madhavajay/dev/PySyft/packages/grid
Success!




In [18]:
# ⚔️ Runnable Code
import hagrid
hagrid.check(['localhost:8081'], silent=True, wait=True)

Checking...


┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━┓
┃ PyGrid    ┃ Info                                      ┃ 575 ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━┩
│ UI (βeta) │ http://localhost:8081/login               │ ✅  │
│ api       │ http://localhost:8081/api/v1/openapi.json │ ✅  │
└───────────┴───────────────────────────────────────────┴─────┘

In [19]:
# ⚔️ Runnable Code
import syft as sy
domain_client = sy.login(
    port=8081,
    email="info@openmined.org",
    password="changethis"
)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into test_domain... done!


## Pointers

In the above example we can see that the client initially sends two init actions with the self contained values of 1 and 2. There are two ways to get data into `syft`, one is to create it locally and send it over, and the other is to call a remote `constructor` directly.

First lets explicitly create a wrapped `python` 1 which syft understands.

In [22]:
# ⚔️ Runnable Code
one = sy.lib.python.Int(1)
type(one), one

(syft.lib.python.Int, 1)

This `syft` python primitive can be utilized in almost every way as though it was a regular python int, however it also knows how to `serialize` and `deserialize` itself and interact with other `syft` objects. This is an implementation detail which users rarely need to think about.

Now lets send it to the `domain` so it can be used.

In [25]:
# ⚔️ Runnable Code
one_ptr = one.send(domain_client)
one_ptr

<IntPointer -> test_domain:5eaa1771c50b433395d8964e0c7c3cbe>

For illustration purposes we can create the second integer by calling the remote constructor and passing in a literal.

In [26]:
# ⚔️ Runnable Code
two_ptr = domain_client.syft.lib.python.Int(2)
two_ptr

<IntPointer -> test_domain:e9953c65e8a543edb2f62a697bd612c8>

You will note that the domain `client` exposes a series of APIs we call the `ast` which match a list of `allowed` methods and functions on supported data types. However, users rarely need to call constructors on the `ast` directly as one you obtain a pointer, most of the methods or infix `__dunder__` operations you will want to use can be invoked directly using normal python syntax.

These two integer pointers we now have, are mostly identical. In the second instance `syft` detected that the literal `2` is not a syft object and converted it under the hood, and sent it over so that it can be passed into the remote constructor on the `domain`.

Let's take a closer look at some of the methods available on this `IntPointer`:

In [30]:
# ⚔️ Runnable Code
type(one_ptr), dir(one_ptr)

(syft.proxy.syft.lib.python.IntPointer,
 ['__abs__',
  '__add__',
  '__and__',
  '__annotations__',
  '__bool__',
  '__ceil__',
  '__class__',
  '__del__',
  '__delattr__',
  '__dict__',
  '__dir__',
  '__divmod__',
  '__doc__',
  '__eq__',
  '__float__',
  '__floor__',
  '__floordiv__',
  '__format__',
  '__ge__',
  '__getattribute__',
  '__gt__',
  '__hash__',
  '__iadd__',
  '__ifloordiv__',
  '__imod__',
  '__imul__',
  '__init__',
  '__init_subclass__',
  '__invert__',
  '__ipow__',
  '__isub__',
  '__itruediv__',
  '__le__',
  '__lshift__',
  '__lt__',
  '__mod__',
  '__module__',
  '__mul__',
  '__name__',
  '__ne__',
  '__neg__',
  '__new__',
  '__or__',
  '__pos__',
  '__pow__',
  '__radd__',
  '__rand__',
  '__rdivmod__',
  '__reduce__',
  '__reduce_ex__',
  '__repr__',
  '__rfloordiv__',
  '__rlshift__',
  '__rmod__',
  '__rmul__',
  '__ror__',
  '__round__',
  '__rpow__',
  '__rrshift__',
  '__rshift__',
  '__rsub__',
  '__rtruediv__',
  '__rxor__',
  '__setattr__',
  '__si

If you know how python handles things like infix operations with `__dunder__` methods you will recognize that this `Pointer` class has methods like `__add__` and `__mul__`.

Let's re-assigned these new `Pointer` variables to our old vanilla python variables from above.

In [32]:
# ⚔️ Runnable Code
a = one_ptr
b = two_ptr

We should be able to execute the rest of the above `action` graph by simply using normal python syntax locally on our client.

Here we ask our local python interpreter to run `+` against `a` and `b` which finds the `__add__` method on `a` and passes `b` to it, assigning the return value to `c`.

The `pointer` has a whole set of mirror methods that match the real `__add__` and on execution, it will generate a local `uuid`, send an `async` / `WithoutReply` type `action` with the correct data, operations and desired `uuid` key for storing the result, and immediately returns a new `pointer` of an expected return type and that new locally generated `uuid` baked into it.

In [34]:
# ⚔️ Runnable Code
c = a + b

So what is `c`? Since the result of all operations on pointers, are another pointer to the real return type, the variable `c` now holds a `Pointer`, with the `uuid` of the location of the result of the addition between `a` and `b` that will occur on the `domain` as soon as the `action` arrives.

In [35]:
# ⚔️ Runnable Code
c

<IntPointer -> test_domain:35e34822c554417b9a55c418babc9b38>

Since we created `a` and `b`, the intersection of permissions given to `c` will allow us to `get` the result.

In [36]:
# ⚔️ Runnable Code
c.get(delete_obj=False)

3

Let's continue with the next operation from the previous example.

In [37]:
# ⚔️ Runnable Code
d = c * b

Again we can fetch this result since we have all permissions to read `c` and `b` which `d` was derived from.

In [38]:
# ⚔️ Runnable Code
d.get(delete_obj=False)

6

Let's try some division.

In [65]:
e = d / 2
e

<FloatPointer -> test_domain:056bdad16eab4489b2d898f224dce5d7>

Interesting. We got back a `FloatPointer`. How did `syft` know the result of `integer` division is a `float`?

## Under the hood

You may ask where does all this automatic binding of objects and their remote procedure call come from? Inside `syft` there are several areas where we define the mappings between what constructors and methods can be called in actions and their return types.

One such set of mappings for the python primitives is found here:

```python
# packages/syft/src/syft/lib/python/__init__.py

methods = [
    ("syft.lib.python.Int.__add__", "syft.lib.python.Int"),
    ("syft.lib.python.Int.__truediv__", "syft.lib.python.Float"),
    ("syft.lib.python.Int.__divmod__", "syft.lib.python.Float"),
    ("syft.lib.python.Int.__floordiv__", "syft.lib.python.Float"),
    ("syft.lib.python.Int.__invert__", "syft.lib.python.Int"),
    ("syft.lib.python.Int.__abs__", "syft.lib.python.Int"),
    ("syft.lib.python.Int.__bool__", "syft.lib.python.Bool"),
    ("syft.lib.python.Int.__divmod__", "syft.lib.python.Tuple"),
    ("syft.lib.python.Int.__rdivmod__", "syft.lib.python.Int"),
    ("syft.lib.python.Int.__radd__", "syft.lib.python.Int"),
    ("syft.lib.python.Int.__sub__", "syft.lib.python.Int"),
    ("syft.lib.python.Int.__rsub__", "syft.lib.python.Int"),
```

Let's use this knowledge to manually construct an `action` message to divide the above `d` by `2`.

First lets create a `UID` for the key to store our result under.

In [68]:
# ⚔️ Runnable Code
uid = sy.common.UID()
uid

<UID: fb73d5a86eda4b87b4e4d1eb1f282f24>

Now lets construct the `action` we want, it will require us to supply the following:
- path
- _self
- args
- kwargs
- id_at_location
- address

The `path` is the location of the real `function` which we bind to a set of module paths nested inside the `syft` module itself. If the path does not exist in any of our explicit `allowlist` paths, it will be ignored. This prevents someone from sending an action to something like `os.system` or other potentially harmful functions.

The `_self` is what would be bound to `self` if the function is a method, however some functions are `@staticmethd` or `@classmethod`. The `args` and `kwargs` allow us to pass in anything that would normally be passed in to the function as `*args` or `*kwargs`. If the remote side finds that the input is a `Pointer` itself, it will look up the `UID` of the pointer, fetch it from the key-value store and pass the real value into the function.

The result of any computation will be boxed back up as a `serializable` object and set to the `value` of the key supplied as `id_at_location`. This is how subsequent operations can continue to chain execution on previously generated `UID`s as the graph is resolved.

Finally, we need the `message` to have the `address` of the `domain` we are messaging. If a `node` which is not our target `domain` gets the message first, and it has a known route to the `domain` it will forward the message for us. This is how the `ProxyClient` from the previous session works.

Let's try sending our own `RunClassMethodAction` for division. Note we could go and send in a `2` first and pass a pointer to that `2` as the first `args` but `syft` will do this automatically for us.

In [82]:
# ⚔️ Runnable Code
from syft.core.node.common.action.run_class_method_action import RunClassMethodAction

cmd = RunClassMethodAction(
    path="syft.lib.python.Int.__truediv__",
    _self=d,
    args=[2],
    kwargs={},
    id_at_location=uid,  # normal destination changed
    address=domain_client.address,
)
cmd

RunClassMethodAction IntPointer.__truediv__(int, )

Now we just need to send the `message` to the `domain` by using the correct method on the client like so:

In [83]:
domain_client.send_immediate_msg_without_reply(msg=cmd)

Where's our `pointer`?

The messaging functions are not responsible for `pointer` creation. Pointers are usually synthesized by the code which creates the `action` so we will need to make our own by hand.

Let's take a look at our `ast` definition, here we see that `__truediv__` returns a `float`.

```
("syft.lib.python.Int.__truediv__", "syft.lib.python.Float"),
```

To create a `pointer` we need to know what its type or `path` is and have the `uuid` or `UID` we want to use.

In [84]:
e_ptr = domain_client.lib_ast.query("syft.lib.python.Float").pointer_type(
    client=domain_client, id_at_location=uid
)
e_ptr

<FloatPointer -> test_domain:fb73d5a86eda4b87b4e4d1eb1f282f24>

You will notice that all `pointers` are bound to a particular `client`, this provides them with the context necessary to send messages while keeping the normal `python` method and function syntax exactly the same as if you were operating locally without pointers.

In [85]:
e_ptr.get(delete_obj=False)

[2022-10-13T11:00:16.954537+0100][CRITICAL][logger]][5694] Unable to Get Object with ID <UID: fb73d5a86eda4b87b4e4d1eb1f282f24> from store. Possible dangling Pointer. 'Object not found! for UID: fb73d5a8-6eda-4b87-b4e4-d1eb1f282f24'


UnknownPrivateException: Unable to Get Object with ID <UID: fb73d5a86eda4b87b4e4d1eb1f282f24> from store. Possible dangling Pointer. 'Object not found! for UID: fb73d5a8-6eda-4b87-b4e4-d1eb1f282f24'

Let's see what happens if someone who didn't create the data tries to use it.

In [39]:
# ⚔️ Runnable Code
guest_client = sy.login(
    port=8081
)

Connecting to localhost... done! 	 Logging into test_domain... as GUEST...done!


We can use the `UID` from another pointer to get a new pointer to the same object but with different client permissions.

In [54]:
# ⚔️ Runnable Code
a_uid = a.id_at_location.no_dash
a_uid

'5eaa1771c50b433395d8964e0c7c3cbe'

In [55]:
# ⚔️ Runnable Code
guest_a = guest_client.store[a_uid]

Let's try some addition:

In [57]:
# ⚔️ Runnable Code
x_ptr = guest_a + guest_a

That worked. Okay, but can we get the result?

In [62]:
try:
    x_ptr.get(delete_obj=True)
except Exception:
    print("Nope")
    pass

[2022-10-12T09:56:35.605031+0100][CRITICAL][logger]][5694] You do not have permission to .get() Object with ID: <UID: 66013429a54c4053a4520cf7e303d007> on node test_domain Please submit a request.


Nope


Once we have some data to act on, the following two actions can be `functions` or `methods` which are dependent on the previous inputs. Before execution can take place the `node` must check to see if the dependent input keys are available in the store.

All of the above messages are `async` and require no response from the `domain`. However, since the `client` generated the random `UUID`s in the first place, it is able to `synthesize` pointers at each step allowing the `Data Scientists` python interpreter to chain functions and methods one after the other to build up an Action Graph.

If the `Data Scientist` wants to resolve and `get` the actual results for this computation and they have permission to do so, they can call a `sync` message called `GetObjectAction` on the pointer like so:
    

`x.get()`

Actions Instruction for remote execution which includes optional data and possible UID Pointers and usually results in the result being stored remotely with a UID determined by the User.
Security is provided by storing search and read permission dicts on the StorableObject.
GetObjectAction is the only action which has WithReply and blocks.
Most Actions deal with the store but some like SMPC Actions also require Network calls.
We have no concept of Execution Permissions on Actions which could be exploited.

## Security Model

There are some similarities between the `syft` action permissions and the traditional `UNIX-like` filesystem permissions of `read`, `write` and `execute`.

Normally the creation of data as a result of execution is `read`able by a user in the `UNIX` paradigm, however in `syft` this is somewhat inverted. Instead, `execution` is permitted on non-readable `data` however the result which is always stored remotely out of reach of the `client`, carries with it the intersection of permissions of all input data. This generally means that data derived from `private` data will automatically be `private` and will not be `read`able by the client who created it.

In syft the ability to `write` to a given `UUID` key is prevented if the writer did not create it, to prevent the ability to mutate others data.

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

## Services

On the other hand, `services` provide functionality which generally acts on the `metadata` of the system, are usually only available to priviledged users and utilize more granular composeable `RBAC` style permissions.

Instruction for remote execution which includes optional data and possible UID Pointers and usually results in either no result or a blocking Reply Message with a result.

Security is provided by the `@service_auth` decorator which prevents execution if the user key does not match some list.

Some services are available on different Nodes, such as Domain and Network.

A single @service_auth decorator is often insufficient to express the granularity required for Authorization so many ServiceMessages have their own security code often located in the Database ORM layer.

Services predominantly read and write from the Database.

### Anatomy of a Node

Zoom into a container

### Pointers

- computational graph
- follow a span?

In [1]:
mention SMPC

In [1]:
actions

## Security

### authentication and authorization

### rbac

## Serde

### data vs code

### Pickle

the dangers of serde and pickle

https://sector7.computest.nl/post/2022-08-process-injection-breaking-all-macos-security-layers-with-a-single-vulnerability/

protobuf

zero copy

capnp

recursive serde

#### JS

## Homework

- 

## BONUS

- syckle
- scrollymaze

- steal object / mutation